# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [2]:
import getpass
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers, spark_helpers
import pickle
import random
import numpy as np
import pandas as pd
import functools

ZH_HB_ID = 8503000

%load_ext autoreload
%autoreload 2

## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-fouche>

In [4]:
# init spark session
spark = SparkSession(sc)

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe. We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

# convert to pandas dataframe
pandas_df = metadata.toPandas()

# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

# dump metadata in pickle
with open('./data/metadata.pickle', 'wb') as handle:
    pickle.dump(stations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# load metadata from pickle
with open('./data/metadata.pickle', 'rb') as handle:
    stations = pickle.load(handle)

### Cleaning main dataset

In [7]:
# load full data
raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
# raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [8]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'LINIEN_TEXT':'line',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [9]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [10]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', spark_helpers.get_weekday(df.date))

# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

# there is still 51'571'541 rows in zurich area
# df.count()

# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '2017-12-10 00:00:00')

# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

# discard ill-formated rows where the train leaves a station before arriving in it
df2 = df2.where((df2.schedule_dep >= df2.schedule_arrival) | functions.col('schedule_arrival').isNull() | functions.col('schedule_dep').isNull())

## Modeling the network

### From stops to trips

In [11]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', spark_helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', spark_helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [12]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep', 'line', 'transport_type', 'train_type', 'arr_forecast_status', 'weekday', 'real_arrival']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [13]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))
trips2 = trips2.select("*", functions.lag("arr_forecast_status").over(w).alias("next_arr_forecast_status"))
trips2 = trips2.select("*", functions.lag("real_arrival").over(w).alias("next_real_arrival"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [14]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', spark_helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type, trips2.schedule_dep,trips2.next_sched_arr))

# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [15]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [16]:
regenerate_models = False
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = (helpers.model_network(trips4, date))
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            helpers.network_to_datetime(network) # works inplace
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [17]:
# load the networks from the pickles
models = []
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)
    print(day + ' loaded')

monday loaded
tuesday loaded
wednesday loaded
thursday loaded
friday loaded
saturday loaded
sunday loaded


### Compute walking network

In [18]:
# compute walking network
walking_network = helpers.compute_walking_network(stations)
print('walking network loaded')

walking network loaded


# Pierre

## Prediction / Regression

Deux idées : Le but c'est de prévoir le retard ou l'avance d'un départ ou d'une arrivée pour un arrêt précis à un moment précis et un trip précis.

- Faire une vraie regression ou n'importe quel algo pour déterminer le retard. On utilise comme features : l'arrêt (catégorie), latitude longitude (peut être mettre ensemble en tuple), jour precis ou jour de la semaine ?, le type de transport (bus-train ...), le trip id ou redondant ???, 

- Faire une moyenne, à un arrêt, un trip, un jour, une heure la moyenne de retard qu'il a eu dans ses conditions ... 

On va avoir besoin des trips id dans le dico histoire de pouvoir lier le graph des arrêts avec le réél trajet, quel bus/train fait cet edge.

## Predictive model :

In this part we are going to build a preditive model using historical arrival/departure time data. Thus, our goal is to predict an uncertainty or certainty rate of taking a transport change with success, and finally to spread it for a whole journey using a route planning algorithm explained in the next section.

### 1) Processing the datas :

First of all, we have to group the data that match (same trip between two transport stops at the same schedule), and to compute the delay for the trips where the historical time was measured.

In [19]:
# We drop what we do not need for the prediction
trips5 = trips4.drop('stop_type', 'next_type', 'is_valid', 'arr_forcast_status', 
                     'schedule_time', 'schedule_arrival', 'next_sched_dep', 
                     'next_time', 'next_tid', 'real_arrival', 'arr_forecast_status')

# We refactor the dates by keeping only the time of the day and not the entire day
trips5 = trips5.withColumn('schedule_dep', spark_helpers.keep_time(trips5.schedule_dep))
trips5 = trips5.withColumn('next_sched_arr', spark_helpers.keep_time(trips5.next_sched_arr))
trips5 = trips5.withColumn('next_real_arrival', spark_helpers.keep_time(trips5.next_real_arrival))

# We compute the arrival delay of each stop station
df3 = trips5.withColumn("delay_arrival", 
                     functions.unix_timestamp('next_real_arrival', 'HH:mm:ss') -
                     functions.unix_timestamp('next_sched_arr', 'HH:mm:ss'))

# We create time interval in order to differentiate rush hour and other time of the day
# We have to try different combinations of interval time (that gave best prediction)
df3 = df3.withColumn('arrival_interval', spark_helpers.create_rush(df3.next_sched_arr))
# We also gather the week day in 4 buckets (Wednesnay, Saturday and Sunday separatly and the other days together)
df3 = df3.withColumn('weekday', spark_helpers.group_weekday(df3.next_sched_arr))

# We create a dataframe where we only keep the rows where there is a real arrival time
arrival_df = df3.filter(df3.next_arr_forecast_status == "GESCHAETZT")
arrival_df = arrival_df.drop('next_arr_forecast_status')

# arrival_df.groupby('transport_type').count().show()

As we can see only the trains are concerned with historical time data, thus we were able to compute the delay only for those trips. Moreover we only had 1 millions rows of evaluated time on the 30 millions trips that we have. 

So, one solution was to consider that there is no delay for the other trips (Bus / Tram), thus, there will almost alway have 0% chance to miss a connection between two transports as there are a majority of Bus and Tram in the center of Zurich.

An other solution was to try to predict the delay of those trips according to the ones we already had. That is why we decided to make a linear regression using the localization (latitude, longitude) of the stop, as along with the day and time of the trip.

### 2) Complete the delay data with Linear Regression :

# MATTHIAS A COMPLETER

### 3) Evaluate the uncertainty according to the delay:

Thus for now we have the delay for a majority of trips for a weekday and a time interval. Thus we have to find a way to evaluate the uncertainty according to the delays we have. We decide to use the interquartile range (IQR) in order to manage the outliers. In fact, the interquartile range is the difference between the upper and lower quartiles (75th, 25th percentiles) Q3 and Q2. 

$IQR = Q3 - Q2$

It allows us to mesure the dispersion of the delays for each trip. Moreover we associate it with the interquartile mean (IQM) which is the mean of the data included in the interquartile range and is insensitive to outliers.

$IQM = \frac{2}{n}\sum_{i = \frac{n}{4}+1}^\frac{3n}{4} x_i$

Thus for each trips, for a given day of the week and time interval, we now have an estimation of the worst delay possible without being affected by the outliers.

In [20]:
# We group the trips by their week day and time interval
delay_distribution = arrival_df.groupby(['trip_id', 'arrival_interval', 'weekday']).agg(functions.collect_list(functions.col('delay_arrival'))).alias('distri')
delay_distribution = delay_distribution.withColumn('distri', spark_helpers.delete_neg(functions.col('collect_list(delay_arrival)')))

# We compute the IQM / IQR and worst case
delay_distribution = delay_distribution.withColumn('IQM', spark_helpers.iqm(delay_distribution.distri))
delay_distribution = delay_distribution.withColumn('interquartile', spark_helpers.interquartile(delay_distribution.distri))
delay_distribution = delay_distribution.withColumn('worst_case', delay_distribution.IQM + delay_distribution.interquartile)

#delay_distribution_pd = delay_distribution.toPandas()

# Then we store it in order to easily make our predictions.
regenerate_distri_pickle = False
if regenerate_distri_pickle:
    delay_distribution_pd.to_pickle('./data/delay_distri.pickle')
else:
    delay_distribution_pd = pd.read_pickle('./data/delay_distri.pickle')

delay_distribution_pd

,trip_id,arrival_interval,weekday,collect_list(delay_arrival),distri,IQM,interquartile,worst_case
0,85:11:18231:002,0,0,"[0, -60, 0, 60, 0, 0, -60, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",19.636363636363637,60.0,79.636364
1,85:11:70828:002,0,0,[-180],[0],0.0,0.0,0.000000
2,85:11:88764:003,0,0,"[0, -60, 60, 0, 0, 0]","[0, 0, 60, 0, 0, 0]",0.0,0.0,0.000000
3,85:11:88853:005,1,0,"[60, 0, 60, 0, -60, 0, -60, 0, 0]","[60, 0, 60, 0, 0, 0, 0, 0, 0]",0.0,0.0,0.000000
4,85:11:19536:001,1,0,"[0, 0, 0, 0, 0, 0, 0, -60, 0, 0, 0, 0, -60, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0,0.000000
5,85:11:18330:001,1,0,"[60, 0, 0, 0, 60, 0, -60, 0, 120, 60, 0, 60, 6...","[60, 0, 0, 0, 60, 0, 0, 0, 120, 60, 0, 60, 60,...",24.485294117647058,60.0,84.485294
6,85:11:31432:003,1,0,"[-180, 60, 0, 0, 0, -120, 60, 0, 0, 0]","[0, 60, 0, 0, 0, 0, 60, 0, 0, 0]",0.0,0.0,0.000000
7,85:11:2684:001,1,0,"[-60, -60, 120, 60, -60, 0, -60, -60, 0, 0, -6...","[0, 0, 120, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.285714285714286,60.0,64.285714
8,85:11:70727:008,0,0,[-60],[0],0.0,0.0,0.000000
9,85:11:89538:000,1,0,"[240, 240, 240, 240]","[240, 240, 240, 240]",240.0,0.0,240.000000


## Shortest path algorithm

In [21]:
# example
sp = helpers.shortest_path(models, walking_network, stations,ZH_HB_ID, 8502559, datetime(2018, 1, 15, 14))
helpers.reduced_path_tostring(helpers.reduce_path(sp), stations)

line walk from Zürich HB to Zürich HB SZU 14:00 -> 14:01(1 stops)
line S10 from Zürich HB SZU to Uitikon Waldegg 14:05 -> 14:18(6 stops)
line walk from Uitikon Waldegg to Waldegg, Birmensdorferstrasse 14:18 -> 14:21(1 stops)


## Some tests

In [22]:
# filter out the stations that are not reachable from Zürich HB on mondays
reachable_stations_ids = helpers.get_reachable_stations(models[0], walking_network, ZH_HB_ID)
reachable_stations = {sid: stations[sid] for sid in reachable_stations_ids}

In [23]:
# select two stops at random and create a shortest path. Do that 100 times
shortest_paths = []
n_runs = 100
date = datetime(2018, 3, 12, 14)
for i in range(n_runs):
    if i%10==0:
        print(100*i/n_runs, '% finished')
    #source = random.choice(list(stations.keys()))
    dest = random.choice(list(reachable_stations.keys()))
    shortest_paths.append(helpers.shortest_path(models, walking_network, reachable_stations, ZH_HB_ID, dest, date))

0.0 % finished
10.0 % finished
20.0 % finished
30.0 % finished
40.0 % finished
50.0 % finished
60.0 % finished
70.0 % finished
80.0 % finished
90.0 % finished


## Shortest path with arrival time
To compute the shortest path with arrival time instead of departure time, we implemented a reversed version of our shortest path algorithm

In [24]:
sp = helpers.shortest_path_reverse(models, walking_network, reachable_stations, ZH_HB_ID, 8590727, datetime(2018, 1, 15, 23,5))
helpers.reduced_path_tostring(helpers.reduce_path(sp), stations)

line S15 from Zürich HB to Zürich Hardbrücke 22:22 -> 22:24(1 stops)
line S3 from Zürich Hardbrücke to Schlieren 22:31 -> 22:38(2 stops)
line walk from Schlieren to Schlieren, Bahnhof 22:42 -> 22:44(1 stops)
line 308 from Schlieren, Bahnhof to Schlieren, Talacker-Windegg 22:44 -> 22:45(1 stops)
line walk from Schlieren, Talacker-Windegg to Unterengstringen, Langacher 22:48 -> 23:05(4 stops)


## Determining uncertainty of a trip according to the predictive model

In [28]:
def search_inter(trip_id, weekday, time_interval):
    interquartile = delay_distribution_pd[(delay_distribution_pd.trip_id == str(trip_id)) & 
                            (delay_distribution_pd.arrival_interval == str(time_interval)) & 
                            (delay_distribution_pd.weekday == str(weekday))].worst_case
    #print(interquartile)
    return float(interquartile) if interquartile.size != 0 else float(0)


def routing_algo(path):
    prev_edge = path[0]
    certainty = {}
    for i, edge in enumerate(path):
        
        if (edge[4] != prev_edge[4]) and (edge[4] != 'walk') and ((prev_edge[4] != 'walk') or (i-1 != 0)):
            print(edge)
            time_for_change = edge[2] - prev_edge[3]
            
            if (prev_edge[4] == 'walk') and (i-1 != 0):      
                time_inter = spark_helpers.rush_inter(path[i-2][3].time().isoformat())
                inter = search_inter(path[i-2][4], spark_helpers.group_weekday_py(path[i-2][3].weekday()), time_inter)
            else:
                time_inter = spark_helpers.rush_inter(prev_edge[3].time().isoformat())
                inter = search_inter(prev_edge[4], spark_helpers.group_weekday_py(edge[3].weekday()), time_inter)
            
            #print(time_for_change, time_inter, inter)
            
            if inter:
                certainty[i-1] = min(time_for_change.total_seconds() / inter, 1)
            else:
                certainty[i-1] = 1
        prev_edge = edge
        
        if certainty:
            certainty_tot = functools.reduce((lambda v1,v2: v1*v2), certainty.values())
        else:
            certainty_tot = 1
            
    return certainty, certainty_tot

In [29]:
routing_algo(sp)

(8503020, 8503001, datetime.datetime(2018, 1, 15, 22, 31), datetime.datetime(2018, 1, 15, 22, 35), '85:11:18384:001', 'S3')
(8590786, 8590802, datetime.datetime(2018, 1, 15, 22, 44), datetime.datetime(2018, 1, 15, 22, 45), '85:849:301867-32301-1', '308')


({0: 1, 3: 0.0}, 0.0)